# Germline Analysis Blueprint 

This blueprint shows how to run a standard Germline workflow on whole exome data from `fastq` to `vcf`. We will start by downloading the sample reads and a set of BWA-indexed reference files. Then we will align the reads to the reference using BWA-MEM and call the variant using DeepVariant. The diagram below outlines these steps. 

![fq2bam_diagram](images/pbworkflow.png)


To process the exome samples, we will use a GPU accelerated software suite called [Parabricks](https://docs.nvidia.com/clara/parabricks/latest/index.html). It contains over 25 popular secondary analysis tools for manipulating DNA and RNA data, including tools for alignment (BWA, Giraffe, Minimap2, STAR, etc.), variant calling (DeepVariant, HaplotypeCaller, DeepSomatic, StarFusion, etc.), and post processing steps such as quality checks and gvcf processing. Each tool offers nearly identical output to the CPU versions, but with speedups of up to 100x. For the full list of tools, see the [documentation](https://docs.nvidia.com/clara/parabricks/latest/toolreference.html). 

In this blueprint, we are running inside the Parabricks Docker container which can be found in the [NVIDIA GPU Container (NGC) Registry](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/clara/containers/clara-parabricks). 

## Download the dataset

The data set used in this lab is the whole exome sample NA12878 from the [NIH](https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data_indexes/NA12878/sequence.index.NA12878_Illumina_HiSeq_Exome_Garvan_fastq_09252015) sequenced on Illumina. The fastq files for this sample, as well as the HG38 reference files (already indexed) can be downloaded using the `download_data.sh` script. 

In [ ]:
!sh scripts/download_data.sh 

The data directory already exists.


It can take up to 15 minutes to download and organize the data into the correct structure for this blueprint. In the meantime, let's discuss the data being downloaded. 

When the script completes, the `data` directory should contain two `.fastq.gz` files as a `ref` folder containing all the reference files, and an empty `output` folder where the output files from this workflow will be stored.  

```
data
├── NIST7035_TAAGGCGA_L001_R1_001.fastq.gz
├── NIST7035_TAAGGCGA_L001_R2_001.fastq.gz
├── output
└── ref
    ├── Homo_sapiens_assembly38.dict
    ├── Homo_sapiens_assembly38.fasta
    ├── Homo_sapiens_assembly38.fasta.amb
    ├── Homo_sapiens_assembly38.fasta.ann
    ├── Homo_sapiens_assembly38.fasta.bwt
    ├── Homo_sapiens_assembly38.fasta.fai
    ├── Homo_sapiens_assembly38.fasta.pac
    ├── Homo_sapiens_assembly38.fasta.sa
    ├── Homo_sapiens_assembly38.known_indels.vcf.gz
    └── Homo_sapiens_assembly38.known_indels.vcf.gz.tbi
```

TODO: More data details

Let's verify that the data downloaded correctly by running `ls` on the `data` directory. 

In [8]:
! ls data

NIST7035_TAAGGCGA_L001_R1_001.fastq.gz	output
NIST7035_TAAGGCGA_L001_R2_001.fastq.gz	ref


Let's verify that the reference files downloaded correctly by running `ls` on the `ref` folder. 

In [8]:
! ls data/ref

Homo_sapiens_assembly38.dict
Homo_sapiens_assembly38.fasta
Homo_sapiens_assembly38.fasta.amb
Homo_sapiens_assembly38.fasta.ann
Homo_sapiens_assembly38.fasta.bwt
Homo_sapiens_assembly38.fasta.fai
Homo_sapiens_assembly38.fasta.pac
Homo_sapiens_assembly38.fasta.sa
Homo_sapiens_assembly38.known_indels.vcf.gz
Homo_sapiens_assembly38.known_indels.vcf.gz.tbi


Now the data is downloaded and organized for downstream analysis. 

## Align the fastq files

In this section, we will use the [Parabricks fq2bam](https://docs.nvidia.com/clara/parabricks/latest/documentation/tooldocs/man_fq2bam.html) tool to align our `.fastq` files to the reference. The code to do this is contained in the bash script located at `scripts/fq2bam.sh`. The following cell executes this code. It will take a few minutes to run, so kick it off now and read below about what the script is doing. 

In [ ]:
!sh scripts/fq2bam.sh

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation



[Parabricks Options Mesg]: Checking argument compatibility
[Parabricks Options Mesg]: Set --bwa-options="-K #" to produce compatible pair-ended results with previous versions of
fq2bam or BWA MEM.
[Parabricks Options Mesg]: Automatically generating ID prefix
[Parabricks Options Mesg]: Read group created for /root/germline-blueprint/data/NIST7035_TAAGGCGA_L001_R1_001.fastq.gz
and /root/germline-blueprint/data/NIST7035_TAAGGCGA_L001_R2_001.fastq.gz
[Parabricks Options Mesg]: @RG\tID:H7AP8ADXX.1\tLB:lib1\tPL:bar\tSM:sample\tPU:H7AP8ADXX.1
[PB Info 2025-Mar-04 00:19:04] ------------------------------------------------------------------------------
[PB Info 2025-Mar-04 00:19:04] ||                 Parabricks accelerated Genomics Pipeline                 ||
[PB Info 2025-Mar-04 00:19:04] ||                              Version 4.4.0-1                             ||
[PB Info 2025-Mar-04 00:19:04] ||          

Parabricks fq2bam is a wrapper for [BWA-MEM](https://github.com/lh3/bwa) that is optimized to run on the GPU, providing up to 60x speedups compared to the CPU-only version. It will also sort the output and can be configured to mark duplicates and recalibrate base quality scores. Below is a diagram showing how the steps within fq2bam are connected. 

![fq2bam_diagram](images/fq2bam.png)

Now let's see how to run fq2bam by opening up `scripts/fq2bam.sh`. We have pasted the contents of this file below. 

```code
#!/bin/bash 

REF="data/ref/Homo_sapiens_assembly38.fasta"
KNOWN_SITES="data/ref/Homo_sapiens_assembly38.known_indels.vcf.gz"
FASTQ_1="data/NIST7035_TAAGGCGA_L001_R1_001.fastq.gz"
FASTQ_2="data/NIST7035_TAAGGCGA_L001_R2_001.fastq.gz"
OUT_BAM="data/output/NIST7035_TAAGGCGA_L001_R1_001.bam"
OUT_RECAL="data/output/recal.txt"

pbrun fq2bam \
    --ref ${REF} \
    --in-fq ${FASTQ_1} ${FASTQ_2} \
    --knownSites ${KNOWN_SITES} \
    --out-bam ${OUT_BAM} \
    --out-recal-file ${OUT_RECAL} \
    --gpusort --gpuwrite 
```

The beginning of this script defines paths to the files needed to run fq2bam. For this example we will provide a reference `.fasta`, known sites `.vcf`, and the two reads as `.fastq` files. We will also needs output paths for the resulting `.bam` containing the aligned reads and the `recal.txt` with the recalibration scores.  

At the end of of this script is the run command. Every command in Parabricks starts with `pbrun` followed by the name of the tool to run and then any arguments. Most of the arguments are file inputs as defined above, however there are two boolean flags at the end that will improve the performance of the tool. 

The coordinate sorting step of alignment can be moved to the GPU by including `--gpusort`. The GPU can also help writing the output `.bam` file by enabling the `--gpuwrite` flag. 

In this example, we are running with a minimal set of arguments but there are dozens of extra options outlined in the [documentation](https://docs.nvidia.com/clara/parabricks/latest/documentation/tooldocs/man_fq2bam.html#fq2bam-reference). 

Once the cell executing fq2bam has finished running, we can look inside the `data/output` folder and see the that `.bam` file containing our aligned reads has been generated. 

In [ ]:
! ls data/output | grep .bam

NIST7035_TAAGGCGA_L001_R1_001.bam
NIST7035_TAAGGCGA_L001_R1_001.bam.bai



## Run variant calling

In this section, we will use the [Parabricks DeepVariant](https://docs.nvidia.com/clara/parabricks/latest/documentation/tooldocs/man_deepvariant.html) tool to call variants on the `.bam` we generated using fq2bam.  The code to do this is contained in the bash script located at `scripts/deepvariant.sh`. The following cell executes this code. It will take around 10 minutes to run, so kick it off now and read below about what the script is doing. 

In [ ]:
!sh scripts/deepvariant.sh

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation

Detected 1 CUDA Capable device(s), considering 1 device(s)
  CUDA Driver Version / Runtime Version          12.4 / 12.3
Using model for CUDA Capability Major/Minor version number:    89
/usr/local/parabricks/binaries/bin/deepvariant /root/germline-blueprint/data/ref/Homo_sapiens_assembly38.fasta /root/germline-blueprint/data/NIST7035_TAAGGCGA_L001_R1_001.bam 1 2 -o /root/germline-blueprint/data/NIST7035_TAAGGCGA_L001_R1_001.vcf -n 6 --model /usr/local/parabricks/binaries/model/80+/shortread/deepvariant_wes.eng --channel_insert_size --pileup_image_width 221 --max_reads_per_partition 1500 --partition_size 1000 --vsc_min_count_snps 2 --vsc_min_count_indels 2 --vsc_min_fraction_snps 0.12 --min_mapping_quality 5 --min_base_quality 10 --alt_aligned_pileup none --variant_caller VERY_SENSITIVE_CALLER --dbg_min_base_quality 15 --ws_min_windows_distance 80 --aux_fields_to_keep HP --p_error 0.001 --max_ins_size 10


DeepVariant is a variant caller that uses a convolutional neural network (CNN) as opposed to bayesian statistics to discover variants. This method often performs better, especially on low frequency variants. It has the added benefit that it can be retrained to improve performance on any dataset. In this blueprint, we will use the [GPU optimized version of DeepVariant found in Parabricks](https://docs.nvidia.com/clara/parabricks/latest/documentation/tooldocs/man_deepvariant.html).  

Now let's see how to run fq2bam by opening up `scripts/deepvariant.sh`. We have pasted the contents of this file below. 

```
#!/bin/bash 

REF="data/ref/Homo_sapiens_assembly38.fasta"
IN_BAM="data/output/NIST7035_TAAGGCGA_L001_R1_001.bam"
OUT_VCF="data/output/NIST7035_TAAGGCGA_L001_R1_001.vcf"

pbrun deepvariant \
    --ref ${REF} \
    --in-bam ${IN_BAM} \
    --out-variants ${OUT_VCF} \
    --use-wes-model \
    --run-parition 
```

Just like the fq2bam script, the beginning of the script defines the file paths needed to run Parabricks DeepVariant. This time we only need the reference `.fasta`, the aligned reads `.bam` file generated by fq2bam in the previous section, and the path to the output `.vcf` to store the variants. 

At the end of the script is the run command. Just as before, it starts with `pbrun` but this time instead of `fq2bam` we are running `deepvariant` with a minimal set of arguments. The file inputs are familiar, but let's look into the two additional flags at the end. 

DeepVariant is based on a CNN model. Different CNNs can be trained on different datasets. By default, Parabricks DeepVariant uses a model trained on whole genome (WGS) data, however in this blueprint we are using an exome and must therefore use a CNN trained on whole exome (WES) data. To do this, we use the `--use-wes-model` argument. 

To further optimize the performance, we include `--run-parition` which will partition the exome and run each partition on a different process, thus improving performance. 

For more information on all the flags, check out the [documentation](https://docs.nvidia.com/clara/parabricks/latest/documentation/tooldocs/man_deepvariant.html#deepvariant-reference). 

Once the cell executing deepvariant has finished running, we can look inside the `data/output` folder and see the that `.vcf` file containing the variants has been generated. 

In [ ]:
! ls data/output | grep .vcf

NIST7035_TAAGGCGA_L001_R1_001.vcf


## Next Steps

 TODO: CTA